# Value-at-risk

---

VaR is a measure of portfolio risk. For instance, a 1% VaR of -5% means that there is a 1% chance of earning a portfolio return of less than -5%. Think of it as a (lower) percentile or quantile of a portfolio returns distribution, i.e., we are concerned about the tail risk — the small chance of losing a remarkably large portfolio value. Such a large loss is funded by our own funds, i.e., capital which is an expensive source of funding compared to other peoples’ funds, i.e., debt. Therefore the estimation of VaR and similar market risk management measures inform banks and insurance firms with regards to the levels of capital they need to hold in order to have a buffer against unexpected downturns — market risk.

For our purpose, let us begin by fetching a data set of 5 stocks from Yahoo. The stocks are Apple, Google, Microsoft, Intel and Box. We use a daily frequency for our data for the year 2016. We use the stock's daily closing prices to compute the continuously compounded returns: $\log\left(\frac{V_{t+1}}{V_{t}}\right) = \log(V_{t+1}) - \log(V_{t})$.

![](portfolio_returns.png)

Let's estimate the expected returns vector, volatilities vector, correlation and variance-covariance matrices. The variance-covariance matrix is recovered from the estimated volatilities vector and correlation matrix: $\Omega = C \odot \sigma \sigma^{T}$ where $\odot$ is the Hadamard product, $C \in \mathbb{R}^{5 \times 5}$ and $\sigma \in \mathbb{R}^{5 \times 1}$. Portfolio volatility is estimated as: $w^{T}\Omega w$ where $w \in \mathbb{R}^{5 \times 1}$

We consider the 3 major methods used in market risk management, specifically for the estimation of VaR. Please note that there are multiple different methods for estimating VaR and other more coherent risk measures such as Conditional Value-at-Risk (CVaR) however we are only considering the few major ones.


# Installing packages

In [35]:
import sys
!{sys.executable} -m pip install numpy pandas scipy matplotlib sklearn

  Using cached sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)


## VaR: Variance-covariance method

---

The first one is the variance-covariance method and uses the estimated portfolio volatility $w^{T}\Omega w$ under the Gaussian assumption to estimate VaR. Let's assume we are attempting to estimate 5% VaR: This means that there is a 5% probability of obtaining a portfolio return of less than the VaR value. Using the variance-covariance approach the calculation is: $\left[\left(w^{T}\Omega w\right) \mathcal{N}^{-1}(1\%)\right] + w^{T}\mu$, where $\mu \in \mathbb{R}^{5 \times 1}$ is the expected returns vector.

In [1]:
# Import the libraries:Panda (reading data, manipulation, analysis), numpy(for mathematical array operations)

from __future__ import division
import pandas as pd
import numpy as np
from scipy.stats import norm


from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
from sklearn.preprocessing import normalize


In [32]:
# I indicated an arbirary weight allocation to portfolio stocks
weights = np.array([0.166684,0.300577,0.258903,0.227786,0.046049])

# I set the initial investment amount
initial_investment = 2500000

# Opening data
df = pd.read_csv("ret_data.csv")
df

,Apple,Google,Microsoft,Intel,Box
0,-0.025379,0.000997,0.004552,-0.004718,-0.075180
1,-0.019764,0.001400,-0.018332,-0.022419,-0.048452
2,-0.043121,-0.023443,-0.035402,-0.038206,-0.041840
3,0.005274,-0.016546,0.003062,-0.010418,-0.047107
4,0.016063,0.002181,-0.000574,0.017304,-0.009520
...,...,...,...,...,...
246,0.001976,-0.001708,-0.004890,0.001083,0.005096
247,0.006331,0.002074,0.000632,0.002701,0.010116
248,-0.004273,-0.008246,-0.004593,-0.011940,-0.007215
249,-0.000257,-0.002883,-0.001430,0.000819,0.005776


In [33]:
# To explore some key statistics:
df.describe()

,Apple,Google,Microsoft,Intel,Box
count,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.000377,0.000158,0.000501,0.000259,-0.000141
std,0.014773,0.012576,0.014316,0.014426,0.027995
min,-0.067965,-0.054645,-0.074411,-0.095432,-0.125227
25%,-0.005969,-0.004931,-0.006021,-0.006334,-0.011028
50%,0.000756,0.000217,0.000356,0.001343,0.001599
75%,0.007676,0.007688,0.007161,0.008498,0.015230
max,0.062940,0.043293,0.056571,0.034435,0.082065


In [34]:
cov_matrix= df.cov()

cov_matrix

,Apple,Google,Microsoft,Intel,Box
Apple,0.000218,0.000088,0.000104,0.000097,0.000150
Google,0.000088,0.000158,0.000126,0.000089,0.000085
Microsoft,0.000104,0.000126,0.000205,0.000124,0.000136
Intel,0.000097,0.000089,0.000124,0.000208,0.000155
Box,0.000150,0.000085,0.000136,0.000155,0.000784


In [22]:
# eigenvalues of covariance matrixe
np.linalg.eigvalsh(cov_matrix)

array([5.10884557e-05, 9.73317458e-05, 1.23219532e-04, 3.53496287e-04,
       9.48065819e-04])

In [41]:
# Calculating the mean returns for each stock

average_returns= df.mean()

average_returns

Apple        0.000377
Google       0.000158
Microsoft    0.000501
Intel        0.000259
Box         -0.000141
dtype: float64

In [42]:
#Overall portfolio average returns,using the dot product formular for the means against investment weights

Portf_mean =average_returns. dot(weights)

Portf_mean

0.00029243952798192134

In [39]:
# Portfolio Standard Deviation is calculated using :

Portf_stdev= round(np.sqrt(weights.T.dot(cov_matrix).dot(weights)),4)

Portf_stdev

0.0114

In [43]:
#Calculating the mean value of the given investment

Mean_investment = round((1+ Portf_mean)*initial_investment,4)

Mean_investment

2500731.0988

In [44]:
# Standard deviation of the investment

Stdev_investment = initial_investment* Portf_stdev

Stdev_investment

28500.0

In [45]:
# I then proceed to choose the confidence interval of interest. say 95  per cent

Conf_level1 = 0.05

In [46]:
# Using the Scipy percentage point function(ppf) method to generate values for the inverse cummulative density function 
#to a normal distribution, I plugged in the mean and stdev of the portfolio calculated earlier

#It takes a percentage and returns a standard deviation multiplier for what value that percentage occurs at
# For instance, norm.ppf(0.90, loc= 134, scale =3.45) will return a value(that functions as standard-deviation multiplier) marking
# where 95% of data point would be contained if our data follows a normal distribution. 

Cutoff1 =norm.ppf(Conf_level1, Mean_investment, Stdev_investment)

Cutoff1

2453852.7704318827

In [47]:
# Eventually, I am able to calculate the VaR at the confidence interval chosen. 
# Interpretation: By this result, we are saying that the loss in our portfolio will not exceed $46148 over 1-day period @95 C.I

VaR_1dl = round(initial_investment- Cutoff1,3)

VaR_1dl

46147.23

In [48]:
# What if we decided to evaluate this over a bigger window of time? 
# In this case,we take 1-day VaR multiplied by square root of the select time period
# That is, in order to calculate n-Day VaR
# Observation:  As the the time window is expanded as below, the potential loss tends to increase. This seems logical

num_days = 5

for x in range(1, num_days+1):
     print(str(x) + " day VaR @ 95% Confidence: " + str(np.round(VaR_1dl * np.sqrt(x),2)))

1 day VaR @ 95% Confidence: 46147.23
2 day VaR @ 95% Confidence: 65262.04
3 day VaR @ 95% Confidence: 79929.35
4 day VaR @ 95% Confidence: 92294.46
5 day VaR @ 95% Confidence: 103188.34


## VaR: Historical simulation method
The second method is a non-parametric approach where we sample with replacement from the historical data to estimate a portfolio returns distribution. The 5% VaR is simply the appropriate quantile from this sampled portfolio returns distribution.

Historical Simulation (HS) VaR is instead efficient when the risk manager cannot, or doesn’t intend to, make assumptions on the underlying distribution of returns as it is calculated by the simple picking of the chosen percentile loss in a given period of time.

This method is even simpler than the parametric one & that is precisely its weakness. The main underlying logic is that the past is a good predictor for the future. By looking back, it is then possible to incorporate all data points into the risk calculation but this unfortunately leads to too simplistic a model.

In [81]:
Confidence_Interval=0.95

Value_at_Risk = -np.percentile(df,1-Confidence_Interval)
Value_at_Risk

0.12313456457505044

## VaR: Monte Carlo method

---

The third method is Monte Carlo sampling from a multidimensional Gaussian distribution using the aforementioned $mu$ and $\Omega$ parameters. Finally the 5% VaR is simply the appropriate quantile from this sampled portfolio returns distribution.

In general, MCS is a technique that "converts" uncertainty on input variables of a model into **probability distributions**. By combining the distributions and randomly selecting values from them, it recalculates the simulated model many times, to determine the probability of the output.

Historically, this technique was first used by scientists working on the atomic bomb: it was named after Monte Carlo, the Monaco resort town renowned for its casinos.  Since its introduction in World War II, Monte Carlo simulation has been used to model a variety of physical and conceptual systems.

### How does it work?
Monte Carlo simulation performs risk analysis by building models of possible results by *substituting a range of possible input values, that constitute uncertainty, into a statistical distribution*. It then computes possible outcomes repeatedly, each time using a different set of random values from the probability functions that "model" the input. Depending upon the number of random input variables and their distribution, a Monte Carlo simulation could involve thousands or tens of thousands of "rounds" before it is complete. When complete, *Monte Carlo simulation produces distributions of possible outcome values*.

By using probability distributions instead of actual input samples, it is possible to model more accurately uncertainty: different choices of distributions will yield different outputs.

### A brief summary of the Monte Carlo Simulation (MCS) technique

- A MCS allows several inputs to be used at the same time to compute the probability distribution of one or more outputs
- Different types of probability distributions can be assigned to the inputs of the model, depending on any *a-priori* information that is available. When the distribution is completely unknown, a common technique is to use a distribution computed by finding the best fit to the data you have
- The MCS method is also called a **stochastic method** because it uses random variables. Note also that the general assumption is for input random variables to be independent from each other. When this is not the case, there are techniques to account for correlation between random variables.
- A MCS generates the output as a range instead of a fixed value and shows how likely the output value is to occur in that range. In other words, the model outputs a probability distribution.

### Common distributions used in MCS
In what follows, we summarize the most common probability distributions that are used as *a-priori* distributions for input random variables:

- *Normal/Gaussian Distribution*: this is a continuous distribution applied in situations where the mean and the standard deviation of a given input variable are given, and the mean represents the most probable value of the variable. In other words, values "near" the mean are most likely to occur.  This is symmetric distribution, and it is not bounded in its co-domain. It is very often used to  describe natural phenomena, such as people’s heights, inflation rates, energy prices, and so on and so forth. An illustration of a normal distribution is given below:
![normal_distribution](https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Normal_Distribution_PDF.svg/320px-Normal_Distribution_PDF.svg.png)

- *Lognormal Distribution*: this is a distribution which is appropriate for variables taking values in the range $[0, \infty]$. Values are positively skewed, not symmetric like a normal distribution.  Examples of variables described by some lognormal distributions include, for example, real estate property values, stock prices, and oil reserves. An illustration of a lognormal distribution is given below:
![log_normal_distribution](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ae/PDF-log_normal_distributions.svg/320px-PDF-log_normal_distributions.svg.png) 

- *Triangular Distribution*: this is a continuous distribution with fixed minimum and maximum values. It is bounded by the minimum and maximum values and can be either symmetrical (the most probable value = mean = median) or asymmetrical. Values around the most likely value (e.g. the mean) are more likely to occur.  Variables that could be described by a triangular distribution include, for example, past sales history per unit of time and inventory levels. An illustration of a triangular distribution is given below:
![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Triangular_distribution_PMF.png/320px-Triangular_distribution_PMF.png)

- *Uniform Distribution*: this is a continuous distribution bounded by known minimum and maximum values. In contrast to the triangular distribution, the likelihood of occurrence of the values between the minimum and maximum is the same. In other words, all values have an equal chance of occurring, and the distribution is simply characterized by the minimum and maximum values. Examples of variables that can be described by a uniform distribution include manufacturing costs or future sales revenues for a new product. An illustration of the uniform distribution is given below:
![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/96/Uniform_Distribution_PDF_SVG.svg/320px-Uniform_Distribution_PDF_SVG.svg.png)

- *Exponential Distribution*: this is a continuous distribution used to model the time that pass between independent occurrences, provided that the rate of occurrences is known. An example of the exponential distribution is given below:
![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Exponential_pdf.svg/320px-Exponential_pdf.svg.png)

- *Discrete Distribution* : for this kind of distribution, the "user" defines specific values that may occur and the likelihood of each of them.  An example might be the results of a lawsuit: 20% chance of positive verdict, 30% change of negative verdict, 40% chance of settlement, and 10% chance of mistrial.


### Pseudo-Code for using Monte Carlo to calculate VaR and Expected Shortfall (ES)

* $n=5$ is the number of assets.
* $m=5000$ is the number of samples/simulations I carried out.
* $T=10$ is the $10$ days time horizon.
* $M=251$ be the number of days I look back in for each asset to compute mean and std. 
* Let $S_i(0)$ be the today's price of the $i^{th}$ asset in the portfolio where $i\in\{1,\dots,n\}$.
* Let $c_i = 100$ be the number of contracts of $i^{th}$ asset we have in our portfolio. In our example, they are all identical but in reality, they can be different depending on your position. 
* Let $P_p(0) = \sum_{i=1}^{n} c_iS_i(0)$ be the initial portfolio price.
* Let $w = \{w_i\}_{i\in\{1,\dots,n\}}\in\mathbb{R}^n$ be the weight of EACH asset of the portfolio where $w_i = \frac{c_iS_i(0)}{P_p(0)}$ for $i\in\{1,\dots,n\}$.
* Let $H\in\mathbb{R}^{n\times M}$ be a matrix storing the HISTORICAL data of $M$ days and $n$ assets. 
* Let $$R = \frac{H[:,1:]}{H[:,:-1]} - 1 \in \mathbb{R}^{n\times (M-1)}$$ be a $n$ by $M-1$ RETURNS matrix storing all the returns for $n$ assets where $R_{ij} = \frac{H_{i,j+1}}{H_{ij}}- 1$ and $i\in\{1,\dots,n\}$, $j\in\{1,\dots,M-1\}$.
* Let $\mu\in\mathbb{R}^n$ be the return mean vector for each asset where $$\mu_i = \frac{\sum_{j=1}^{M-1} R_{ij}}{M-1}$$. Note, here I have computed $\textbf{Equally weighted mean}$ but I think it is better to give a higher weightage to more recent return values. Hence, we using $\textbf{Exponentially weighted mean}$ is a good idea which I omitted for simplicity. 
* Let $\Sigma \in \mathbb{R^{n\times n}}$ be the correlation matrix of the returns of assets in the portfolio.
* Let $D = \sqrt{diag(\Sigma)}\in \mathbb{R^{n\times n}}$ be the the diagonal matrix storing standard deviation of each asset where $D_{ii} = \sqrt{\Sigma_{ii}} = \sigma_{i}$.
* Let: 
$$\Sigma = LL^T$$ be a __Cholesky Decomposition__ of correlation matrix and $L$ is a __lower triangular matrix__. 

* $\textbf{For $k = 1:m$}$:  $\quad \quad(\textit{OUTER loop}$)
    - Take $prod^{(k,0)} = \mathbb{1}\in\mathbb{R}^n$ is a $n$ dimensional vector of just ones. I need it in the future. 
    
    - $\textbf{For $j = 1:T$}$: $\quad \quad(\textit{INNER loop}$)
        * Let $X^{(k,j)}\in\mathbb{R}^{n}$ be __independent standard normal vector__ i.e. each $X^{(k,j)}_i\sim N(0,1)$ and $Cov(X^{(k,j)}_i,X^{(k,j)}_p) = 0$ where $i\neq p$. 
        * Let $Y^{(k,j)} = LX^{(k,j)}$ be __CORRELATED standard normal vector__ i.e. each $Y^{(k,j)}_i\sim N(0,1)$ and $Cov(Y^{(k,j)}_i,Y^{(k,j)}_p)\neq 0$ necessarily. 
        * $r^{(k,j)} = \mu + DY^{(k,j)} $ is a $n$ dimensional return vector for storing the return of EACH asset. I have $\textbf{assumed that returns of EACH asset are normally distributed with mean}$ $\mu_i$ and variance $\sigma_i^2$ i.e $r^{(k,j)}_i\sim N(\mu_i, \sigma_i^2)$ and $r^{(k,j)}_i = \mu_i + \sigma_iY^{(k,j)}_i$ for $i\in\{1,\dots,n\}$.  
        * $prod^{(k,j+1)} = prod^{(k,j)}*(\mathbb{1}+r^{(k,j)})$ is a bit like you are stepping through the time $1$ to $T$ and computing the returns of EACH asset by random simulations which is then compounded. 
    - Now we step out of the INNER loop and compute the __portfolio return at the $k^{th}$ outer iteration__: 
    $$P^{(k)}_r = \sum_{i=1}^{n} w_i*prod^{(k,T)}_i \in\mathbb{R}$$.
* Now we step out of the OUTER loop, and take: 
$$\textbf{P}_r = \{P_r^{(1)},\dots,P_r^{(m)}\}\in\mathbb{R}^m$$ be a __$m$ dimensional vector of all the portfolio returns simulated above__. 
* $\text{Loss_vec} = P_p(0)[\textbf{P}_r - \mathbb{1}] \in \mathbb{R}^m$ stores how the __price of the portfolio__ i.e. $P_p(T)$ have changed for today's price to the time horizon $T=10$. 
* SORT the $\text{Loss_vec}$ vector in the ascending order such that:
$$\tilde{P}_r^{(1)} \leq \dots \leq \tilde{P}_r^{(m)}$$
* Take $\alpha = 0.01$ where $(1-\alpha)$ be the confidence interval to which we are computing our VaR to. 
* Take $idx = \lceil m\alpha \rceil$ which is the index at which we have our VaR value. I just use the $\textbf{ceiling}$ fucntion but you can also $\textbf{interpolate}$ between the indices to get a better estimation of the VaR. 
* $VaR = -\tilde{P}_r^{(idx)}$ is the computation of the value at risk at $(1-\alpha)$ confidence interval. 
* $ES = \frac{-1}{idx}\sum_{k=1}^{idx} \tilde{P}_r^{(k)}$ is the expected shortfall which is basically the average of all the returns below and including the VaR. 

In [59]:
'''Here I am setting up the problem by importing all the historic data, 
    computing the mean/std of asset returns and also computing the 
    cholesky decomposition of the correlation matrix etc.'''

no_of_assets = 5; #number of assets
no_of_sims = 5000; #number of simulations.
time_horizon = 10; #time horizon which we are trying to forecast the VaR
go_back_days = 251 # number of days to look at the data in the past to compute the mean/std of asset's returns
alpha = 0.05  # (1-alpha) is the confidence interval we are computing VaR to

tickers = ["Apple","Google","Microsoft","Intel","Box"]  #tickers of the assets in the portfolio
no_of_contracts = [100,100,100,100,100] #number of contracts for EACH asset in the portfolio

# I set the initial investment amount
initial_investment = 2500000

# Opening data
df = pd.read_csv("ret_data.csv")
df

initial_asset_prices = df.iloc[-1] #today prices are the last column
initial_portfolio_price = no_of_contracts@initial_asset_prices #initial portfolio price
weights = no_of_contracts*initial_asset_prices/initial_portfolio_price

mean_vec=df.mean()
mean_vec

Apple        0.000377
Google       0.000158
Microsoft    0.000501
Intel        0.000259
Box         -0.000141
dtype: float64

In [30]:
weights

Apple        0.166684
Google       0.300577
Microsoft    0.258903
Intel        0.227786
Box          0.046049
Name: 250, dtype: float64

In [4]:
std_vec=df.std() #computing the standard deviation
std_vec

Apple        0.014773
Google       0.012576
Microsoft    0.014316
Intel        0.014426
Box          0.027995
dtype: float64

In [51]:
correlation_mat = df.corr()  #computing the correlation matrix
print(correlation_mat)

              Apple    Google  Microsoft     Intel       Box
Apple      1.000000  0.475370   0.489525  0.456646  0.363502
Google     0.475370  1.000000   0.699470  0.489599  0.240516
Microsoft  0.489525  0.699470   1.000000  0.600531  0.338984
Intel      0.456646  0.489599   0.600531  1.000000  0.384221
Box        0.363502  0.240516   0.338984  0.384221  1.000000


In [52]:
# eigenvalues of correlation matrixe
np.linalg.eigvalsh(correlation_mat)

array([0.27777327, 0.481286  , 0.56685801, 0.8238901 , 2.85019261])

In [25]:
#cholesky of the correlation matrix
cholesky_mat = np.linalg.cholesky(correlation_mat) 
cholesky_mat

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.47536993, 0.87978601, 0.        , 0.        , 0.        ],
       [0.4895251 , 0.53054297, 0.69201831, 0.        , 0.        ],
       [0.45664608, 0.30976126, 0.30728922, 0.77530359, 0.        ],
       [0.36350176, 0.07697136, 0.17370091, 0.18187691, 0.89369495]])

In [60]:
def mc_VaR_standard_way(printing=False):
    '''This is a the code for my pseudo-code above'''
    portfolio_returns_sims = np.zeros(no_of_sims) #creating an empty zero array

    for k in range(no_of_sims):
        prod_k = np.ones(no_of_assets) #vector of ones of no_of_assets dims
        np.random.seed(k) #fixed a seed 
        for j in range(time_horizon):
            X_kj = np.random.normal(loc = 0.0, scale = 1.0, size=no_of_assets) #generating standard normal sims
            Y_kj = cholesky_mat@X_kj  #correlating them with each other

            # making an assumption that returns of each asset is normally distributed 
            # with mean mu_i and standard deviation simga_i, hence I unstandarise it.
            r_kj = mean_vec + std_vec*Y_kj   
            prod_k = prod_k*(1+r_kj)  #a bit like I am stepping through the time horizon and doing compound interest
        portfolio_returns_sims[k] = weights@prod_k  #simulated returns of the portfolio in the kth sim

    loss_vector = initial_portfolio_price*(portfolio_returns_sims - 1) #change in the portfolio price at t= 0  and t=T
    sorted_loss_vector = np.sort(loss_vector) #sorting the loss vector
    #finding the index where we have VaR value
    #I just took the ceiling function here but feel free to interpolate here
    index = int(np.ceil(alpha*no_of_sims)) - 1 
    full_VaR = sorted_loss_vector[index]  #computing the VaR 
    ES = sum(sorted_loss_vector[:index+1])/(index+1) #computing the Expected Shortfall (ES)

    if printing:
        print(f'\n- Weights of EACH asset in the portfolio: {weights}')
        print(f'- Initial Price of portfolio: $ {initial_portfolio_price}')
        print(f'- Returns mean for EACH asset: {mean_vec}')
        print(f'- Returns standard deviation for EACH asset: {std_vec}\n')
        print(f'- Correlation Matrix:\n{correlation_mat}\n')
        print(f'- Cholesky of correlation Matrix:\n{cholesky_mat}\n')
        print(f'- {time_horizon} days VaR with {(1-alpha)*100}% CI: $ {-full_VaR}')
        print(f'- {time_horizon} days Expected Shortfall (ES) with {(1-alpha)*100}% CI: $ {-ES}')
    
    return full_VaR

full_Var = mc_VaR_standard_way(printing=True)
print('='*100)


- Weights of EACH asset in the portfolio: Apple        0.166684
Google       0.300577
Microsoft    0.258903
Intel        0.227786
Box          0.046049
Name: 250, dtype: float64
- Initial Price of portfolio: $ -4.6953169280287295
- Returns mean for EACH asset: Apple        0.000377
Google       0.000158
Microsoft    0.000501
Intel        0.000259
Box         -0.000141
dtype: float64
- Returns standard deviation for EACH asset: Apple        0.014773
Google       0.012576
Microsoft    0.014316
Intel        0.014426
Box          0.027995
dtype: float64

- Correlation Matrix:
              Apple    Google  Microsoft     Intel       Box
Apple      1.000000  0.475370   0.489525  0.456646  0.363502
Google     0.475370  1.000000   0.699470  0.489599  0.240516
Microsoft  0.489525  0.699470   1.000000  0.600531  0.338984
Intel      0.456646  0.489599   0.600531  1.000000  0.384221
Box        0.363502  0.240516   0.338984  0.384221  1.000000

- Cholesky of correlation Matrix:
[[1.         0.    

### Simpler Case Monte Carlo: Computing one day VaR and then extending it to T days by multiplying by $\sqrt{T}$

This is also the MC method, but I have assumed that returns of EACH assets are I.I.D with respect to EACH day. Hence, scaling by a factor of $\sqrt{T}$ (as Brownian Motion is of the order $O(\sqrt{T}))$. 

In [61]:
def mc_VaR_simpler(printing=False):
    portfolio_returns_sims = np.zeros(no_of_sims) #creating an empty zero array

    for k in range(no_of_sims):
        np.random.seed(k) #fixed a seed 
        X_k = np.random.normal(loc = 0.0, scale = 1.0, size=no_of_assets) #generating standard normal sims
        Y_k = cholesky_mat@X_k  #correlating them with each other and Y_k is also standard normal.

        #making an assumption that returns of each asset is normally distributed 
        # with mean mu_i and standard deviation sigma_i, hence I unstandarise it
        r_k = mean_vec + std_vec*Y_k   
        portfolio_returns_sims[k] = weights@(1+r_k)  #simulated returns of portfolio in the kth sim

    loss_vector = initial_portfolio_price*(portfolio_returns_sims-1)
    sorted_loss_vector = np.sort(loss_vector)
    index = int(np.ceil(alpha*no_of_sims)) - 1 #I just took the ceiling function here but feel free to interpolate here
    one_day_VaR = sorted_loss_vector[index] #computing the one-day-VaR here
    full_VaR = one_day_VaR*np.sqrt(time_horizon) #extending to a T day VaR

    ES = np.sqrt(time_horizon)*sum(sorted_loss_vector[:index+1])/(index+1) #expected shortfall
    
    if printing:
        print(f'\n- Weights of EACH asset in the portfolio: {weights}')
        print(f'- Initial Price of portfolio: $ {initial_portfolio_price}')
        print(f'- Returns mean for EACH asset: {mean_vec}')
        print(f'- Returns standard deviation for EACH asset: {std_vec}\n')
        print(f'- Correlation Matrix:\n{correlation_mat}\n')
        print(f'- Cholesky of correlation Matrix:\n{cholesky_mat}\n')
        print(f'- {time_horizon} days VaR with {(1-alpha)*100}% CI: $ {-full_VaR}')
        print(f'- {time_horizon} days Expected Shortfall (ES) with {(1-alpha)*100}% CI: $ {-ES}')
        
    return full_VaR, one_day_VaR
        
full_Var = mc_VaR_simpler(printing=True)
print('='*100)


- Weights of EACH asset in the portfolio: Apple        0.166684
Google       0.300577
Microsoft    0.258903
Intel        0.227786
Box          0.046049
Name: 250, dtype: float64
- Initial Price of portfolio: $ -4.6953169280287295
- Returns mean for EACH asset: Apple        0.000377
Google       0.000158
Microsoft    0.000501
Intel        0.000259
Box         -0.000141
dtype: float64
- Returns standard deviation for EACH asset: Apple        0.014773
Google       0.012576
Microsoft    0.014316
Intel        0.014426
Box          0.027995
dtype: float64

- Correlation Matrix:
              Apple    Google  Microsoft     Intel       Box
Apple      1.000000  0.475370   0.489525  0.456646  0.363502
Google     0.475370  1.000000   0.699470  0.489599  0.240516
Microsoft  0.489525  0.699470   1.000000  0.600531  0.338984
Intel      0.456646  0.489599   0.600531  1.000000  0.384221
Box        0.363502  0.240516   0.338984  0.384221  1.000000

- Cholesky of correlation Matrix:
[[1.         0.    